In [214]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import cv2
import os
import keras.backend as K
from keras.losses import binary_crossentropy

In [215]:
TRAIN_PATH = "/kaggle/input/airbus-ship-detection/train_v2/"
TEST_PATH = "/kaggle/input/airbus-ship-detection/test_v2/"

LABELS_PATH = "/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv"
SUBMISSION_PATH = "/kaggle/input/airbus-ship-detection/sample_submission_v2.csv"

In [216]:
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def get_multimask(rle_code_list, img_shape=(768,768)):
    multimask = np.zeros(img_shape)
    for rle in rle_code_list:
        if isinstance(rle, str):
            multimask += rle_decode(rle)
    return multimask

In [217]:
def plot_image_mask(image, rles):
    multimask = get_multimask(rles)
    multimasked_img = image.copy()
    multimasked_img[multimask==1] = (255, 0, 255)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (20, 5))
    ax1.imshow(image)
    ax1.set_title('Origin')
    ax2.imshow(multimask)
    ax2.set_title('Mask')
    ax3.imshow(multimasked_img)
    ax3.set_title("Masked Image")
    
    
img_path = os.path.join(TRAIN_PATH, "00021ddc3.jpg")
image = cv2.imread(img_path)

labels_df = pd.read_csv(LABELS_PATH)
image_rles = labels_df[labels_df["ImageId"] == "00021ddc3.jpg"]["EncodedPixels"]
plot_image_mask(image, image_rles)

# Train

In [218]:
labels_df = pd.read_csv(LABELS_PATH)
labels_df["HasShip"] = labels_df["EncodedPixels"].notnull().astype(int)
unique_imgs = labels_df.groupby("ImageId").sum().rename({"HasShip": "TotalShips"}, axis=1)

total = len(unique_imgs)
img_noship = (unique_imgs["TotalShips"] == 0).sum()
img_ship = total - img_noship

print(f"Total: {total}")
print(f"Number of images with ships: {img_ship}")
print(f"Number of images without ships: {img_noship}")
print(f"Ship/NoShip ratio: {img_ship / img_noship}")
(unique_imgs[unique_imgs["TotalShips"] != 0]["TotalShips"].value_counts() / img_ship).plot(kind="bar", title="Num of ships distribution");

In [219]:
from sklearn.model_selection import train_test_split
train_ids, valid_ids = train_test_split(unique_imgs,
                 test_size = 0.3,
                 stratify = unique_imgs['TotalShips'])
train_df = pd.merge(labels_df, train_ids, left_on="ImageId", right_on="ImageId")
valid_df = pd.merge(labels_df, valid_ids, left_on="ImageId", right_on="ImageId")
print(train_df.shape[0], 'training masks')
print(valid_df.shape[0], 'validation masks')

# Classification Model

In [220]:
class ClassificationGenerator(tf.keras.utils.Sequence):
    def __init__(self, unique_ids, labels, batch_size,  img_newsize=(224, 224)):
        self.unique_ids = unique_ids
        self.labels = labels
        self.batch_size = batch_size
        self.img_newsize = img_newsize
        
    def __len__(self):
        return int(np.floor(len(self.unique_ids) / self.batch_size))
    
    def __getitem__(self, index):
        batch_ids = self.unique_ids[index * self.batch_size:(index+1) * self.batch_size]
        batch_X = np.array([self.__read_img(x) for x in batch_ids])
        batch_Y = self.labels[index * self.batch_size:(index+1) * self.batch_size].values
        return batch_X, batch_Y
    
    def __read_img(self, img_id):
        img = cv2.imread(os.path.join(TRAIN_PATH, img_id))
        img = cv2.resize(img, self.img_newsize, interpolation= cv2.INTER_LINEAR)
        return img

In [221]:
from tensorflow.keras.applications import EfficientNetB0

def build_classification_model():
    inputs = keras.layers.Input(shape=(224,224,3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    model.trainable = False
    x = keras.layers.GlobalAveragePooling2D()(model.output)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-2),
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [222]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler()
cls_train_df = train_df.drop_duplicates("ImageId")
cls_train_df, _ = ros.fit_resample(cls_train_df, cls_train_df["HasShip"])

cls_valid_df = valid_df.drop_duplicates("ImageId")

cls_training_generator = ClassificationGenerator(cls_train_df["ImageId"],
                                                 cls_train_df["HasShip"],
                                                 batch_size=64)
cls_validation_generator = ClassificationGenerator(cls_valid_df["ImageId"],
                                                   cls_valid_df["HasShip"],
                                                   batch_size=64)

In [223]:
cls_model = build_classification_model()
cls_history = cls_model.fit_generator(generator=cls_training_generator,
                                      validation_data=cls_validation_generator,
                                      epochs=5)

In [224]:
cls_model.save("cls_model.h5")

In [225]:
plt.plot(cls_history.history["loss"]);
plt.plot(cls_history.history["val_loss"])
plt.title("Binary Crossentropy Loss");

In [226]:
plt.plot(cls_history.history["accuracy"]);
plt.plot(cls_history.history["val_accuracy"])
plt.title("Accuracy");

# Segmentation Model

In [227]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)


def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)


def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)


def conv_block(layer, num_filters):
    x = keras.layers.Conv2D(num_filters, 3, padding="same")(layer)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    x = keras.layers.Conv2D(num_filters, 3, padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    return x


def encoder_block(layer, num_filters):
    x = conv_block(layer, num_filters)
    p = keras.layers.MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(layer, skip_features, num_filters):
    x = keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(layer)
    x = keras.layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape):
    inputs = keras.layers.Input(input_shape)

    s1, p1 = encoder_block(inputs, 8)
    s2, p2 = encoder_block(p1, 16)
    s3, p3 = encoder_block(p2, 32)
    s4, p4 = encoder_block(p3, 64)

    b1 = conv_block(p4, 128)

    d1 = decoder_block(b1, s4, 64)
    d2 = decoder_block(d1, s3, 32)
    d3 = decoder_block(d2, s2, 16)
    d4 = decoder_block(d3, s1, 8)

    outputs = keras.layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = keras.Model(inputs, outputs, name="U-Net")
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])
    return model

In [228]:
unet_model = build_unet((768,768,3))

In [229]:
class SegmentationGenerator(tf.keras.utils.Sequence):
    def __init__(self, unique_ids, labels, batch_size):
        self.unique_ids = unique_ids
        self.labels = labels
        self.batch_size = batch_size
        
    def __len__(self):
        return int(np.floor(len(self.unique_ids) / self.batch_size))
    
    def __getitem__(self, index):
        batch_ids = self.unique_ids[index * self.batch_size:(index+1) * self.batch_size]
        batch_X = np.array([cv2.imread(os.path.join(TRAIN_PATH, x)) for x in batch_ids])
        batch_Y = np.array([np.expand_dims(np.stack(get_multimask(self.labels[self.labels["ImageId"]==x]['EncodedPixels'].values), 0), -1)
                            for x in batch_ids])
        return batch_X, batch_Y

In [230]:
segm_train_df = train_df[train_df["HasShip"] != 0].drop_duplicates("ImageId")
segm_valid_df = valid_df.drop_duplicates("ImageId")

segm_training_generator = SegmentationGenerator(segm_train_df["ImageId"],
                               segm_train_df, batch_size=8)
segm_validation_generator = SegmentationGenerator(segm_valid_df["ImageId"][:10000],
                               segm_valid_df, batch_size=8)

In [231]:
unet_history = unet_model.fit_generator(generator=segm_training_generator,
                                        validation_data=segm_validation_generator,
                                        epochs=5)

In [232]:
plt.plot(unet_history.history["dice_coef"]);
plt.plot(unet_history.history["val_dice_coef"])
plt.title("Dice Coef");

In [233]:
unet_model.save("./unet.h5")

# Prediction

In [234]:
img_id = segm_valid_df["ImageId"].unique()[6]
img_path = os.path.join(TRAIN_PATH, img_id)
image = cv2.imread(img_path)

labels_df = pd.read_csv(LABELS_PATH)
image_rles = labels_df[labels_df["ImageId"] == img_id]["EncodedPixels"]
plot_image_mask(image, image_rles)

In [235]:
cls_img = cv2.resize(image, (224, 224), interpolation= cv2.INTER_LINEAR)
cls_model.predict(np.array([cls_img]))

In [236]:
plt.imshow(unet_model.predict(np.array([image]))[0]);

In [237]:
unet_model.evaluate(segm_validation_generator)

### Combined Prediction

In [244]:
def predict(inputs , csl_model, segm_model):
    cls_img = cv2.resize(inputs, (224, 224), interpolation= cv2.INTER_LINEAR)
    cls_prediction = cls_model.predict(np.array([cls_img]))[0][0]
    if cls_prediction > 0.5:
        return segm_model.predict(np.array([inputs]))[0] > 0.8
    else:
        return np.zeros(inputs.shape)

In [243]:
plt.imshow(predict(image, cls_model, unet_model));